In [2]:
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, FOAF, RDFS, OWL
import pandas as pd
from pyvis.network import Network
from rdflib import Graph as RDFGraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph

In [3]:
sample = pd.read_csv('sample.csv', low_memory=False)
sample.dropna(inplace=True)
sample.head()

,SourceIP,DestinationIP,SourcePort,DestinationPort,TimeStamp,Duration,FlowBytesSent,FlowSentRate,FlowBytesReceived,FlowReceivedRate,...,PacketTimeCoefficientofVariation,ResponseTimeTimeVariance,ResponseTimeTimeStandardDeviation,ResponseTimeTimeMean,ResponseTimeTimeMedian,ResponseTimeTimeMode,ResponseTimeTimeSkewFromMedian,ResponseTimeTimeSkewFromMode,ResponseTimeTimeCoefficientofVariation,DoH
0,72.21.91.42,192.168.20.191,443.0,51041.0,14/01/2020 15:49,4.544040,11256331.0,2.477164e+06,159324.0,35062.191350,...,0.580949,1.110000e-07,0.000333,0.000027,0.000016,0.000014,0.096659,0.038228,12.454823,False
1,192.168.20.191,195.201.169.48,51043.0,443.0,14/01/2020 15:49,8.171041,930.0,1.138166e+02,11936.0,1460.768585,...,1.745138,3.224052e+00,1.795565,0.886579,0.103698,0.000900,1.308024,0.493259,2.025273,False
2,192.168.20.191,96.17.115.57,51021.0,443.0,14/01/2020 15:49,0.033776,162.0,4.796305e+03,138.0,4085.741355,...,0.500002,0.000000e+00,0.000000,0.033683,0.033683,0.033683,-10.000000,-10.000000,0.000000,False
3,192.168.20.191,96.114.14.140,50308.0,443.0,14/01/2020 15:49,0.060723,108.0,1.778568e+03,120.0,1976.186947,...,0.577350,0.000000e+00,0.000000,0.060687,0.060687,0.060687,-10.000000,-10.000000,0.000000,False
4,192.168.20.191,23.78.199.198,50983.0,443.0,14/01/2020 15:49,0.019545,55.0,2.814019e+03,66.0,3376.822717,...,1.000000,0.000000e+00,0.000000,0.019545,0.019545,0.019545,-10.000000,-10.000000,0.000000,False


In [4]:
print(sample.loc[0].index[0], " ",sample.loc[0][0])
print(sample.loc[0].index[1], " ",sample.loc[0][1])


SourceIP   72.21.91.42
DestinationIP   192.168.20.191


In [5]:
print(Literal(sample.loc[0][1]))

192.168.20.191


In [14]:
predicats = list(sample.columns)
print(predicats)
print(sample.loc[0][predicat[0]])

['SourceIP', 'DestinationIP', 'SourcePort', 'DestinationPort', 'TimeStamp', 'Duration', 'FlowBytesSent', 'FlowSentRate', 'FlowBytesReceived', 'FlowReceivedRate', 'PacketLengthVariance', 'PacketLengthStandardDeviation', 'PacketLengthMean', 'PacketLengthMedian', 'PacketLengthMode', 'PacketLengthSkewFromMedian', 'PacketLengthSkewFromMode', 'PacketLengthCoefficientofVariation', 'PacketTimeVariance', 'PacketTimeStandardDeviation', 'PacketTimeMean', 'PacketTimeMedian', 'PacketTimeMode', 'PacketTimeSkewFromMedian', 'PacketTimeSkewFromMode', 'PacketTimeCoefficientofVariation', 'ResponseTimeTimeVariance', 'ResponseTimeTimeStandardDeviation', 'ResponseTimeTimeMean', 'ResponseTimeTimeMedian', 'ResponseTimeTimeMode', 'ResponseTimeTimeSkewFromMedian', 'ResponseTimeTimeSkewFromMode', 'ResponseTimeTimeCoefficientofVariation', 'DoH']
72.21.91.42


# BASED ON THIS, I WILL ATTEMPT TO CREATE RDF TRIPLES

In [15]:
for i in sample.index:
    for j in sample.columns:
        print(i, " ",j, " ", sample.loc[i][j])
    print("\n")



0   SourceIP   72.21.91.42
0   DestinationIP   192.168.20.191
0   SourcePort   443.0
0   DestinationPort   51041.0
0   TimeStamp   14/01/2020 15:49
0   Duration   4.54404
0   FlowBytesSent   11256331.0
0   FlowSentRate   2477163.713
0   FlowBytesReceived   159324.0
0   FlowReceivedRate   35062.19135
0   PacketLengthVariance   427496.3947
0   PacketLengthStandardDeviation   653.8320845
0   PacketLengthMean   1106.811615
0   PacketLengthMedian   1514.0
0   PacketLengthMode   1514.0
0   PacketLengthSkewFromMedian   -1.86831632
0   PacketLengthSkewFromMode   -0.622772107
0   PacketLengthCoefficientofVariation   0.590734751
0   PacketTimeVariance   1.720110358
0   PacketTimeStandardDeviation   1.311529778
0   PacketTimeMean   2.257565677
0   PacketTimeMedian   2.2460455
0   PacketTimeMode   0.136072
0   PacketTimeSkewFromMedian   0.026351312
0   PacketTimeSkewFromMode   1.617571871
0   PacketTimeCoefficientofVariation   0.58094867
0   ResponseTimeTimeVariance   1.11e-07
0   ResponseTimeTime

# Initializing graph variable

In [8]:
g = Graph()
ex = Namespace("http://example.org/")
g.bind("ex", ex)

In [10]:
for i in sample.index:
    for j in range(len(predicat)):
        g.add((URIRef(ex + f"{i}"), URIRef(ex +predicat[j]), Literal(sample.loc[i][j])))

In [11]:
print(g.serialize(format="turtle"))

@prefix ex: <http://example.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:0 ex:DestinationIP "192.168.20.191" ;
    ex:DestinationPort 5.1041e+04 ;
    ex:DoH false ;
    ex:Duration 4.54404e+00 ;
    ex:FlowBytesReceived 1.59324e+05 ;
    ex:FlowBytesSent 1.125633e+07 ;
    ex:FlowReceivedRate 3.506219e+04 ;
    ex:FlowSentRate 2.477164e+06 ;
    ex:PacketLengthCoefficientofVariation 5.907348e-01 ;
    ex:PacketLengthMean 1.106812e+03 ;
    ex:PacketLengthMedian 1.514e+03 ;
    ex:PacketLengthMode 1.514e+03 ;
    ex:PacketLengthSkewFromMedian -1.868316e+00 ;
    ex:PacketLengthSkewFromMode -6.227721e-01 ;
    ex:PacketLengthStandardDeviation 6.538321e+02 ;
    ex:PacketLengthVariance 4.274964e+05 ;
    ex:PacketTimeCoefficientofVariation 5.809487e-01 ;
    ex:PacketTimeMean 2.257566e+00 ;
    ex:PacketTimeMedian 2.246046e+00 ;
    ex:PacketTimeMode 1.36072e-01 ;
    ex:PacketTimeSkewFromMedian 2.635131e-02 ;
    ex:PacketTimeSkewFromMode 1.617572e+00 ;
    ex:PacketTi

In [12]:
G = rdflib_to_networkx_graph(g)
print("networkx Graph loaded successfully with length {}".format(len(G)))
net = Network('1920px', '1080px')
net.from_nx(G)
net.show("net.html", notebook=False)

networkx Graph loaded successfully with length 169
net.html
